# Spatial ATAC-RNA Seq Mouse Brain

- **Creator**: Sebastian Birk (<sebastian.birk@helmholtz-munich.de>)
- **Date of Creation:** 22.01.2023
- **Date of Last Modification:** 22.12.2024

- In order to run this notebook, a trained model needs to be stored under ```../artifacts/{dataset}/models/{model_label}/{load_timestamp}```.
- Load timestamp: ```13092023_174230```

## 1. Setup

### 1.1 Import Libraries

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("../../utils")

In [ ]:
import argparse
import gc
import os
import random
import shutil
import warnings
from datetime import datetime

import anndata as ad
import matplotlib
import matplotlib.pyplot as plt
import mlflow
import networkx as nx
import numpy as np
import pandas as pd
import scanpy as sc
import scipy.sparse as sp
import scipy.stats as stats
import seaborn as sns
import squidpy as sq
import torch
from matplotlib import gridspec
from matplotlib.pyplot import rc_context
from pywaffle import Waffle
from sklearn.preprocessing import MinMaxScaler

from nichecompass.models import NicheCompass
from nichecompass.utils import (add_gps_from_gp_dict_to_adata,
                                aggregate_obsp_matrix_per_cell_type,
                                create_cell_type_chord_plot_from_df,
                                create_new_color_dict,
                                compute_communication_gp_network,
                                generate_enriched_gp_info_plots,
                                visualize_communication_gp_network)

from analysis_utils import (add_cell_type_latent_cluster_emphasis,
                            add_sub_cell_type,
                            compute_cell_type_latent_clusters,
                            generate_gp_info_plots,
                            plot_category_in_latent_and_physical_space,
                            plot_cell_type_latent_clusters,
                            plot_latent,
                            plot_physical_latent_for_cell_types,
                            sankey,
                            store_top_gps_summary)

### 1.2 Define Parameters

In [ ]:
dataset = "spatial_atac_rna_seq_mouse_brain_batch2"

#### 1.2.1 Generic Parameters

In [ ]:
## Model
# AnnData keys
gp_names_key = "nichecompass_gp_names"
active_gp_names_key = "nichecompass_active_gp_names"
latent_key = "nichecompass_latent"

#### 1.2.2 Dataset-specific Parameters

In [ ]:
load_timestamp = "13092023_174230"
model_label = "reference"
cell_type_key = "celltype"
sample_key = "batch"
samples = ["p22"]
latent_leiden_resolution = 0.3
spot_size = 30
    
latent_cluster_key = f"latent_leiden_{str(latent_leiden_resolution)}"

### 1.3 Run Notebook Setup

In [ ]:
sc.set_figure_params(figsize=(6, 6))
# sc.set_figure_params(dpi=300)
sns.set_style("whitegrid", {'axes.grid' : False})

In [ ]:
# Ignore future warnings and user warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)
warnings.simplefilter(action="ignore", category=RuntimeWarning)

In [ ]:
plt.rcParams['font.family'] = 'Helvetica'
plt.rcParams['font.size'] = 5

### 1.4 Configure Paths and Create Directories

In [ ]:
# Define paths
figure_folder_path = f"../../artifacts/{dataset}/figures/{model_label}/{load_timestamp}"
model_folder_path = f"../../artifacts/{dataset}/models/{model_label}/{load_timestamp}"
result_folder_path = f"../../artifacts/{dataset}/results/{model_label}/{load_timestamp}"
srt_data_folder_path = "../../datasets/st_data" # spatially resolved transcriptomics data
srt_data_gold_folder_path = f"{srt_data_folder_path}/gold"

# Create required directories
os.makedirs(figure_folder_path, exist_ok=True)
os.makedirs(result_folder_path, exist_ok=True)

## 2. Model

In [ ]:
model = NicheCompass.load(dir_path=model_folder_path,
                          adata=None,
                          adata_file_name=f"{dataset}_{model_label}.h5ad",
                          adata_atac=None,
                          adata_atac_file_name=f"{dataset}_{model_label}_atac.h5ad",
                          gp_names_key=gp_names_key)

In [ ]:
model.adata.uns[gp_names_key] = np.array([gp for gp in model.adata.uns[gp_names_key] if not "Add-on " in gp])
model.adata.uns[active_gp_names_key] = np.array([gp for gp in model.adata.uns[active_gp_names_key] if not "Add-on " in gp])
model.adata.uns[gp_names_key] = np.array([gp.replace(" ", "_") for gp in model.adata.uns[gp_names_key]])
model.adata.uns[active_gp_names_key] = np.array([gp.replace(" ", "_") for gp in model.adata.uns[active_gp_names_key]])

In [ ]:
# Dataset-specific metadata
cell_type_df = pd.read_csv(f"{srt_data_gold_folder_path}/spatial_atac_rna_seq_mouse_brain_batch2_cell_type_annotations.csv", index_col=0)
cell_type_df.rename({"predicted.celltype": "celltype"}, axis=1, inplace=True)
cell_type_df.drop("ATAC_clusters", axis=1, inplace=True)
model.adata.obs = model.adata.obs.merge(cell_type_df, left_index=True, right_index=True, how="left")

In [ ]:
# Check number of active gene programs
active_gps = model.get_active_gps()
print(f"Number of total gene programs: {len(model.adata.uns[gp_names_key])}.")
print(f"Number of active gene programs: {len(active_gps)}.")

gp_summary_df = model.get_gp_summary()
gp_summary_df[gp_summary_df["gp_active"] == True][20:]

samples = model.adata.obs[sample_key].unique().tolist()
model.add_active_gp_scores_to_obs()

## 3. Analysis

### 3.1 Create Figures

In [ ]:
### Supplementary Fig. 24a: Sparse marker gene expression ###
adata_all_genes = sc.read_h5ad(f"{srt_data_gold_folder_path}/spatial_atac_rna_seq_mouse_brain_batch2.h5ad")
fig, axs = plt.subplots(nrows=2,
                        ncols=2,
                        figsize=(15, 15))

# Lmo4 TF & Target Gene Peaks
color_map = "RdPu"
sc.pl.spatial(
    adata=adata_all_genes,
    color="Drd3",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Drd3",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)

plt.show()
fig.savefig(f"{figure_folder_path}/drd3_expression.svg",
            bbox_inches="tight")
del(adata_all_genes)

In [ ]:
### Supplementary Fig. 24b: Niches ###
sc.tl.leiden(adata=model.adata,
             resolution=latent_leiden_resolution,
             key_added=latent_cluster_key,
             neighbors_key=latent_key)
sc.tl.leiden(adata=model.adata,
             resolution=0.05,
             key_added=latent_cluster_key,
             restrict_to=(latent_cluster_key, ["7"]),
             neighbors_key=latent_key)
sc.tl.leiden(adata=model.adata,
             resolution=0.05,
             key_added=latent_cluster_key,
             restrict_to=(latent_cluster_key, ["8"]),
             neighbors_key=latent_key)
sc.tl.leiden(adata=model.adata,
             resolution=0.05,
             key_added=latent_cluster_key,
             restrict_to=(latent_cluster_key, ["6"]),
             neighbors_key=latent_key)

niche_annotation_dict = {
    "0":"Sox5/Taf1+ L5-L6",
    "1":"Foxa1/Id3+ L6a-L6b",
    "2":"Meis2/Ebf1+ STR_ACB_core",
    "3":"Hlf/Neurod2+ L1_L4_vasc_olg",
    "4":"Addon16/Irf8+ STR_caudoputamen_lateral",
    "5":"Nr1i3/Lmx1a+ L1-L4",
    "6,0":"Pax2/L-Glutamine+ L5-L6",
    "6,1":"Zbtb7a/Myf5+ L5",
    "7,0":"Gfi1b/Ep300+ STR_caudoputamen_medial",
    "7,1":"Neurog3/Nkx2-2+ Anterior_commisure",
    "8,0":"Pitx3/Cadm1+ STR_ACB_shell",
    "8,1":"Usf2/Lmo4+ islm_Major_island_of_Calleja",
    "9":"Nkx6-2/Nfya+ Corpus_callosum",
    "10":"Plxna2/Hoxb1+ L1-L3",
    "11":"Smad3/Col1a2+ Meninges",
    "12":"Klf10/Crx+ Lateral_septum_complex",
    "13":"Lmo2/Nkx6-1+ Lateral_septal_nucleus",
    "14":"L5-L6",
    "15":"Dach1/Pitx3+ STR_medial",
    "16":"Bcl11a/Brca1+ Lateral_ventricle"}

model.adata.obs["niche"] = model.adata.obs[latent_cluster_key].map(niche_annotation_dict)

latent_cluster_colors = create_new_color_dict(
    adata=model.adata,
    cat_key=latent_cluster_key)
latent_cluster_colors["7,0"] = "#7d87b9"
latent_cluster_colors["4"] = "#4a6fe3"
latent_cluster_colors["2"] = "#023fa5"
latent_cluster_colors["1"] = "#DAB6C4"
latent_cluster_colors["15"] = "#9EB9F3"
latent_cluster_colors["7,1"] = "#9D88A2"
latent_cluster_colors["13"] = "#FF9CDA"
latent_cluster_colors["8,1"] = "#CE5A57"
niche_colors = {niche: latent_cluster_colors[cluster] for cluster, niche in niche_annotation_dict.items()}

model.adata.obs["niche_abr"] = model.adata.obs["niche"].apply(lambda x: x.split("GP+ ")[1].replace(" ", "\n") if "GP+ " in x else x)

niche_abreviated_colors = {}
for key, value in niche_colors.items():
    if "GP+ " in key:
        niche_abreviated_colors[key.split("GP+ ")[1].replace(" ", "\n")] = value
    else:
        niche_abreviated_colors[key] = value
        
save_fig = True
file_path = f"{figure_folder_path}/res_{latent_leiden_resolution}_" \
            "niches_latent_physical_space.svg"

plot_category_in_latent_and_physical_space(
    adata=model.adata,
    plot_label="Niches",
    cat_key="niche",
    groups=None,
    sample_key=sample_key,
    samples=samples,
    cat_colors=niche_colors,
    size=(720000 / len(model.adata)),
    spot_size=spot_size,
    save_fig=save_fig,
    file_path=file_path)

In [ ]:
### Supplementary Fig. 24c: Tissue niche hierarchy ###
sc.tl.dendrogram(adata=model.adata,
                 use_rep="nichecompass_latent",
                 linkage_method="ward",
                 groupby="niche")

fig, (ax) = plt.subplots(1, 1, figsize=(0.6, 2))
sc.pl.dendrogram(
    adata=model.adata,
    groupby="niche",
    orientation="left",
    ax=ax,
    save="_nichecompass_latent.svg")
plt.show()
if os.path.exists(f"{figure_folder_path}/dendrogram_nichecompass_latent.svg"):
    os.remove(f"{figure_folder_path}/dendrogram_nichecompass_latent.svg")
shutil.move(f"./figures/dendrogram_nichecompass_latent.svg", figure_folder_path)
shutil.rmtree("./figures/")

NameError: name 'sc' is not defined

In [ ]:
### Supplementary Fig. 24d: Niche-enriched gene programs ###
gp_resign = True # True

gps = [ 
    "Foxa1_TF_target_genes_GP", "Id3_TF_target_genes_GP",
    "Plxna2_ligand_receptor_target_gene_GP", "Hoxb1_TF_target_genes_GP",
    "Sox5_TF_target_genes_GP", "Taf1_TF_target_genes_GP", # Xanthine_metabolite_enzyme_sensor_GP    
    "Zbtb7a_TF_target_genes_GP", "Myf5_TF_target_genes_GP",
    "Pax2_TF_target_genes_GP", "L-Glutamine_metabolite_enzyme_sensor_GP",
    "Nr1i3_TF_target_genes_GP", "Lmx1a_TF_target_genes_GP",
    "Hlf_TF_target_genes_GP", "Neurod2_TF_target_genes_GP",
    "Add-on_16_GP", "Irf8_TF_target_genes_GP",
    "Meis2_TF_target_genes_GP", "Ebf1_TF_target_genes_GP",
    "Gfi1b_TF_target_genes_GP", "Ep300_TF_target_genes_GP",
    "Nkx6-2_TF_target_genes_GP", "Nfya_TF_target_genes_GP",
    "Neurog3_TF_target_genes_GP", "Nkx2-2_TF_target_genes_GP",
    "Lmo2_TF_target_genes_GP", "Nkx6-1_TF_target_genes_GP",
    "Klf10_TF_target_genes_GP", "Crx_TF_target_genes_GP",
    "Smad3_TF_target_genes_GP", "Col1a2_ligand_receptor_target_gene_GP",
    "Bcl11a_TF_target_genes_GP", "Brca1_TF_target_genes_GP", # Draxin_ligand_receptor_target_gene_GP
    "Usf2_TF_target_genes_GP", "Lmo4_TF_target_genes_GP",
    "Pitx3_TF_target_genes_GP", "Cadm1_ligand_receptor_target_gene_GP",
    "Dach1_TF_target_genes_GP", "Pitx3_TF_target_genes_GP",
]

resign_gps = [
    "Taf1_TF_target_genes_GP",
    "Zbtb7a_TF_target_genes_GP",
    "Meis2_TF_target_genes_GP",
    "Ebf1_TF_target_genes_GP",
    "Neurod2_TF_target_genes_GP",
    "Nr1i3_TF_target_genes_GP",
    "Hlf_TF_target_genes_GP",
    "L-Glutamine_metabolite_enzyme_sensor_GP",
    "Gfi1b_TF_target_genes_GP",
    "Ep300_TF_target_genes_GP",
    "Pitx3_TF_target_genes_GP",
    "Cadm1_ligand_receptor_target_gene_GP",
    "Usf2_TF_target_genes_GP",
    "Lmo4_TF_target_genes_GP",
    "Rps19_ligand_receptor_target_gene_GP",
    "Nfya_TF_target_genes_GP",
    "Nkx6-2_TF_target_genes_GP",
    "Lmo2_TF_target_genes_GP",
    "Pitx3_TF_target_genes_GP",
    "Brca1_TF_target_genes_GP",
    "Bcl11a_TF_target_genes_GP",
    "Nkx6-1_TF_target_genes_GP",
    "Klf10_TF_target_genes_GP",
    "Dach1_TF_target_genes_GP",
    "Pitx3_TF_target_genes_GP"]

if gp_resign:
    for gp in resign_gps:
        model.adata.obs[gp] = -1 * model.adata.obs[gp]

df = model.adata.obs[["niche"] + gps].groupby("niche").mean()
df = df.reindex(model.adata.uns["dendrogram_niche"]["categories_ordered"][::-1])

scaler = MinMaxScaler()
normalized_columns = scaler.fit_transform(df)
normalized_df = pd.DataFrame(normalized_columns, columns=df.columns)
normalized_df.index = df.index
normalized_df.columns = [col.split("_lig")[0].split("_met")[0].split("_TF")[0].replace("Add-on_16_GP", "Addon16").replace("_acid", " acid") + " GP" for col in normalized_df.columns]

plt.figure(figsize=(16, 6))  # Set the figure size
ax = sns.heatmap(normalized_df,
            cmap='viridis',
            annot=False,
            linewidths=0)
plt.xticks(rotation=45,
           ha="right"
          )
plt.savefig(f"{figure_folder_path}/enriched_gps_heatmap.svg",
            bbox_inches="tight")

plt.figure(figsize=(6, 3))  # Set the figure size
ax = sns.heatmap(normalized_df,
            cmap='viridis',
            annot=False,
            linewidths=0,
            cbar_kws={"orientation": "horizontal", "location": "bottom", "shrink": 0.4})
plt.xticks([])
plt.savefig(f"{figure_folder_path}/heatmap_colorbar.svg",
            bbox_inches="tight")

In [ ]:
### Supplementary Fig. 24e: Niche composition ###
cell_type_colors = create_new_color_dict(
    adata=model.adata,
    color_palette=("cell_type_30"),
    cat_key=cell_type_key)
cell_type_colors["CA3GL"] = "#1f77b4"
cell_type_colors["D1MSN"] = "#ff7f0e"
cell_type_colors["LSXGA"] = "#279e68"
cell_type_colors["MXD"] = "#d62728"
cell_type_colors["STRGA"] = "#aa40fc"

tmp = pd.crosstab(model.adata.obs["niche"], model.adata.obs[cell_type_key], normalize='index')
tmp = tmp.reindex(model.adata.uns["dendrogram_niche"]["categories_ordered"][::])
ax = tmp.plot.barh(color=cell_type_colors, stacked=True, figsize=(4, 6)).legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
plt.xlabel("Cell Type Proportions")
plt.savefig(f"{figure_folder_path}/niche_cell_type_proportions.svg", bbox_inches='tight')

In [ ]:
### Supplementary Fig. 24f: Niche composition ###
fig, ax = plt.subplots(figsize=(17, 21))
sankey(
    x=model.adata.obs[cell_type_key],
    y=model.adata.obs["niche_abr"],
    title="Cell Type to Niche Mapping",
    title_left="Cell Type",
    title_right="Niche",
    ax=ax,
    fontsize="16",  # "xx-small",
    #left_order=model.adata.obs[cell_type_key].unique().tolist(),
    colors=cell_type_colors,
    alpha=0.5)
plt.tight_layout()
plt.savefig(f"{figure_folder_path}/sankey_cell_type_to_niche.svg")
plt.show()

In [ ]:
### Supplementary Fig. 25a: Major Island of Calleja niche: enriched prior GPs ###
fig, axs = plt.subplots(nrows=6,
                        ncols=2,
                        figsize=(25, 25))
color_map = "RdGy_r"
sc.pl.spatial(
    adata=model.adata,
    color="Usf2_TF_target_genes_GP",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Usf2 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)    
color_map = "RdPu"
sc.pl.spatial(
    adata=model.adata,
    color="Prkar2b",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Prkar2b",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 1],
    show=False)
sc.pl.spatial(
    adata=model.adata,
    color="Apc",
    use_raw=False,
    color_map=color_map,
    spot_size=spot_size,
    title=f"Apc",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 0],
    show=False)
sc.pl.spatial(
    adata=model.adata,
    color="Pou2f1",
    use_raw=False,
    color_map=color_map,
    spot_size=spot_size,
    title="Pou2f1",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 1],
    show=False)
sc.pl.spatial(
    adata=model.adata_atac,
    color="chr18:34247310-34248119",
    use_raw=False,
    color_map=color_map,
    spot_size=spot_size,
    title=f"chr18:34247310-34248119",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[2, 0],
    show=False)
sc.pl.spatial(
    adata=model.adata_atac,
    color="chr1:165934747-165935645",
    use_raw=False,
    color_map=color_map,
    spot_size=spot_size,
    title=f"chr1:165934747-165935645",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[2, 1],
    show=False)    
plt.show()
fig.savefig(f"{figure_folder_path}/usf2_gp_genes.svg",
            bbox_inches="tight")

fig, axs = plt.subplots(nrows=6,
                        ncols=2,
                        figsize=(25, 25))
color_map = "RdGy_r"
sc.pl.spatial(
    adata=model.adata,
    color="Cebpz_TF_target_genes_GP",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Cebpz GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)    
color_map = "RdPu"
sc.pl.spatial(
    adata=model.adata,
    color="Cebpz",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Cebpz",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[0, 1],
    show=False)
sc.pl.spatial(
    adata=model.adata,
    color="Col11a1",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Col11a1",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[1, 0],
    show=False)
sc.pl.spatial(
    adata=model.adata,
    color="Apc",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Apc",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[1, 1],
    show=False)
sc.pl.spatial(
    adata=model.adata_atac,
    color="chr3:114030022-114030924",
    color_map=color_map,
    spot_size=spot_size,
    title=f"chr3:114030022-114030924",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[2, 0],
    show=False)
sc.pl.spatial(
    adata=model.adata_atac,
    color="chr18:34247310-34248119",
    color_map=color_map,
    spot_size=spot_size,
    title=f"chr18:34247310-34248119",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[2, 1],
    show=False)
sc.pl.spatial(
    adata=model.adata_atac,
    color="chr11:53300121-53301011",
    color_map=color_map,
    spot_size=spot_size,
    title=f"chr11:53300121-53301011",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[3, 0],
    show=False)
plt.show()
fig.savefig(f"{figure_folder_path}/cebpz_gp_genes_peaks.svg",
            bbox_inches="tight")

fig, axs = plt.subplots(nrows=6,
                        ncols=2,
                            figsize=(25, 25))
model.adata.obs["Rela_TF_target_genes_GP_sign_corrected"] = -1 * model.adata.obs["Rela_TF_target_genes_GP"]
color_map = "RdGy_r"
sc.pl.spatial(
    adata=model.adata,
    color="Rela_TF_target_genes_GP_sign_corrected",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Rela GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)    
color_map = "RdPu"
sc.pl.spatial(
    adata=model.adata,
    color="Bclaf1",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Bclaf1",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[0, 1],
    show=False)
sc.pl.spatial(
    adata=model.adata,
    color="Ing4",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Ing4",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[1, 0],
    show=False)
sc.pl.spatial(
    adata=model.adata,
    color="Kmt2a",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Kmt2a",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[1, 1],
    show=False)
sc.pl.spatial(
    adata=model.adata_atac,
    color="chr10:20311949-20312805",
    color_map=color_map,
    spot_size=spot_size,
    title=f"chr10:20311949-20312805",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[2, 0],
    show=False)
plt.show()
fig.savefig(f"{figure_folder_path}/rela_gp_genes_peaks.svg",
            bbox_inches="tight")

fig, axs = plt.subplots(nrows=6,
                        ncols=2,
                        figsize=(25, 25))
model.adata.obs["Rarb_TF_target_genes_GP_sign_corrected"] = -1 * model.adata.obs["Rarb_TF_target_genes_GP"]
color_map = "RdGy_r"
sc.pl.spatial(
    adata=model.adata,
    color="Rarb_TF_target_genes_GP_sign_corrected",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Rarb GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)    
color_map = "RdPu"
sc.pl.spatial(
    adata=model.adata,
    color="Isl1",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Isl1",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[0, 1],
    show=False)
sc.pl.spatial(
    adata=model.adata,
    color="Hap1",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Hap1",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[1, 0],
    show=False)
sc.pl.spatial(
    adata=model.adata_atac,
    color="chr13:116304790-116305726",
    color_map=color_map,
    spot_size=spot_size,
    title=f"chr13:116304790-116305726",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[1, 1],
    show=False)
sc.pl.spatial(
    adata=model.adata_atac,
    color="chr13:116309366-116310250",
    color_map=color_map,
    spot_size=spot_size,
    title=f"chr13:116309366-116310250",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[2, 0],
    show=False)
plt.show()
fig.savefig(f"{figure_folder_path}/rarb_gp_genes_peaks.svg",
            bbox_inches="tight")

fig, axs = plt.subplots(nrows=6,
                        ncols=2,
                        figsize=(25, 25))
model.adata.obs["Rreb1_TF_target_genes_GP_sign_corrected"] = -1 * model.adata.obs["Rreb1_TF_target_genes_GP"]
color_map = "RdGy_r"
sc.pl.spatial(
    adata=model.adata,
    color="Rreb1_TF_target_genes_GP_sign_corrected",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Rreb1 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)    
color_map = "RdPu"
sc.pl.spatial(
    adata=model.adata,
    color="Rreb1",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Rreb1",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 1],
    show=False)
plt.show()
fig.savefig(f"{figure_folder_path}/rreb1_gp_genes_peaks.svg",
            bbox_inches="tight")

fig, axs = plt.subplots(nrows=6,
                        ncols=2,
                        figsize=(25, 25))
model.adata.obs["Gata1_TF_target_genes_GP_sign_corrected"] = -1 * model.adata.obs["Gata1_TF_target_genes_GP"]
color_map = "RdGy_r"
sc.pl.spatial(
    adata=model.adata,
    color="Gata1_TF_target_genes_GP_sign_corrected",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Gata1 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)    
color_map = "RdPu"
sc.pl.spatial(
    adata=model.adata,
    color="Tac1",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Tac1",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[0, 1],
    show=False)
sc.pl.spatial(
    adata=model.adata,
    color="Runx1t1",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Runx1t1",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[1, 0],
    show=False)
sc.pl.spatial(
    adata=model.adata,
    color="Furin",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Furin",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[1, 1],
    show=False)
sc.pl.spatial(
    adata=model.adata_atac,
    color="chr12:104689761-104690616",
    color_map=color_map,
    spot_size=spot_size,
    title=f"chr12:104689761-104690616",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[2, 0],
    show=False)
plt.show()
fig.savefig(f"{figure_folder_path}/Gata1_gp_genes_peaks.svg",
            bbox_inches="tight")

fig, axs = plt.subplots(nrows=6,
                        ncols=2,
                        figsize=(25, 25))
color_map = "RdGy_r"
sc.pl.spatial(
    adata=model.adata,
    color="Lmo4_TF_target_genes_GP",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Lmo4",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)    
color_map = "RdPu"
sc.pl.spatial(
    adata=model.adata,
    color="Lmo4",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Lmo4",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 1],
    show=False)
sc.pl.spatial(
    adata=model.adata_atac,
    color="chr11:70029079-70029942",
    use_raw=False,
    color_map=color_map,
    spot_size=spot_size,
    title=f"chr11:70029079-70029942",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 0],
    show=False)
sc.pl.spatial(
    adata=model.adata_atac,
    color="chr11:70021137-70021886",
    use_raw=False,
    color_map=color_map,
    spot_size=spot_size,
    title="chr11:70021137-70021886",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 1],
    show=False)
plt.show()
fig.savefig(f"{figure_folder_path}/lmo4_gp_genes_peaks.svg",
            bbox_inches="tight")

In [ ]:
### Supplementary Fig. 26b: Major Island Of Calleja: enriched de novo GP ###
fig, axs = plt.subplots(nrows=6,
                        ncols=2,
                        figsize=(25, 25))
model.adata.obs["Add-on_89_GP_sign_corrected"] = -1 * model.adata.obs["Add-on_89_GP"]
color_map = "RdGy_r"
sc.pl.spatial(
    adata=model.adata,
    color="Add-on_89_GP_sign_corrected",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Add-on_89_GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)    
color_map = "RdPu"
sc.pl.spatial(
    adata=model.adata_atac,
    color="chr13:9583665-9584550",
    color_map=color_map,
    spot_size=spot_size,
    title=f"chr13:9583665-9584550",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[0, 1],
    show=False)
sc.pl.spatial(
    adata=model.adata_atac,
    color="chr9:104796453-104797222",
    color_map=color_map,
    spot_size=spot_size,
    title=f"chr9:104796453-104797222",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[1, 0],
    show=False)
sc.pl.spatial(
    adata=model.adata_atac,
    color="chr1:170358844-170359717",
    color_map=color_map,
    spot_size=spot_size,
    title=f"chr1:170358844-170359717",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[1, 1],
    show=False)
sc.pl.spatial(
    adata=model.adata_atac,
    color="chr5:65770160-65771077",
    color_map=color_map,
    spot_size=spot_size,
    title=f"chr5:65770160-65771077",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[2, 0],
    show=False)
sc.pl.spatial(
    adata=model.adata_atac,
    color="chr2:156658586-156659426",
    color_map=color_map,
    spot_size=spot_size,
    title=f"chr2:156658586-156659426",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[2, 1],
    show=False)
sc.pl.spatial(
    adata=model.adata_atac,
    color="chr13:102943643-102944509",
    color_map=color_map,
    spot_size=spot_size,
    title=f"chr13:102943643-102944509",
    use_raw=False,
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[3, 0],
    show=False)
plt.show()
fig.savefig(f"{figure_folder_path}/add-on89_gp_genes_peaks.svg",
            bbox_inches="tight")

In [ ]:
### Supplementary Fig. 26a: Corpus Callosum niche: enriched prior GPs ###
fig, axs = plt.subplots(nrows=6,
                        ncols=2,
                            figsize=(25, 25))
#model.adata.obs["Nfya_TF_target_genes_GP_sign_corrected"] = -1 * model.adata.obs["Nfya_TF_target_genes_GP"]
color_map = "RdGy_r"
sc.pl.spatial(
    adata=model.adata,
    color="Nfya_TF_target_genes_GP",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Nfya GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)    
color_map = "RdPu"
sc.pl.spatial(
    adata=model.adata,
    color="Cldn11",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Cldn11",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[0, 1],
    show=False)
sc.pl.spatial(
    adata=model.adata,
    color="Phgdh",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Phgdh",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[1, 0],
    show=False)
sc.pl.spatial(
    adata=model.adata,
    color="Hmgcs1",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Hmgcs1",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[1, 1],
    show=False)
sc.pl.spatial(
    adata=model.adata,
    color="Fdps",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Fdps",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[2, 0],
    show=False)
sc.pl.spatial(
    adata=model.adata,
    color="Ldlr",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Ldlr",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[2, 1],
    show=False)
plt.show()
fig.savefig(f"{figure_folder_path}/nfya_gp_genes_peaks.svg",
            bbox_inches="tight")

fig, axs = plt.subplots(nrows=6,
                        ncols=2,
                        figsize=(25, 25))
model.adata.obs["Klf6_TF_target_genes_GP_sign_corrected"] = -1 * model.adata.obs["Klf6_TF_target_genes_GP"]
color_map = "RdGy_r"
sc.pl.spatial(
    adata=model.adata,
    color="Klf6_TF_target_genes_GP_sign_corrected",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Klf6 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)    
color_map = "RdPu"
sc.pl.spatial(
    adata=model.adata,
    color="Cers2",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Cers2",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[0, 1],
    show=False)
sc.pl.spatial(
    adata=model.adata,
    color="Gstp2",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Gstp2",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[1, 0],
    show=False)
sc.pl.spatial(
    adata=model.adata,
    color="Gstp1",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Gstp1",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[1, 1],
    show=False)
sc.pl.spatial(
    adata=model.adata,
    color="Tuba1a",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Tuba1a",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[2, 0],
    show=False)
plt.show()
fig.savefig(f"{figure_folder_path}/klf6_gp_genes_peaks.svg",
            bbox_inches="tight")

fig, axs = plt.subplots(nrows=6,
                        ncols=2,
                        figsize=(25, 25))
color_map = "RdGy_r"
sc.pl.spatial(
    adata=model.adata,
    color="Tp73_TF_target_genes_GP",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Tp73 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)    
color_map = "RdPu"
sc.pl.spatial(
    adata=model.adata,
    color="Apod",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Apod",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[0, 1],
    show=False)
sc.pl.spatial(
    adata=model.adata,
    color="Cryab",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Cryab",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[1, 0],
    show=False)
sc.pl.spatial(
    adata=model.adata,
    color="Gstp2",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Gstp2",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[1, 1],
    show=False)
sc.pl.spatial(
    adata=model.adata,
    color="Gstp1",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Gstp1",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[2, 0],
    show=False)
sc.pl.spatial(
    adata=model.adata,
    color="Ybx1",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Ybx1",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[2, 1],
    show=False)
plt.show()
fig.savefig(f"{figure_folder_path}/tp73_gp_genes_peaks.svg",
            bbox_inches="tight")

fig, axs = plt.subplots(nrows=6,
                        ncols=2,
                        figsize=(25, 25))
color_map = "RdGy_r"
sc.pl.spatial(
    adata=model.adata,
    color="Rps19_ligand_receptor_target_gene_GP",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Rps19 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)    
color_map = "RdPu"
sc.pl.spatial(
    adata=model.adata,
    color="Rps19",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Rps19",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[0, 1],
    show=False)
sc.pl.spatial(
    adata=model.adata,
    color="Cd82",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Cd82",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[1, 0],
    show=False)
sc.pl.spatial(
    adata=model.adata,
    color="Scd2",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Scd2",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[1, 1],
    show=False)
sc.pl.spatial(
    adata=model.adata,
    color="Ndrg1",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Ndrg1",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[2, 0],
    show=False)
sc.pl.spatial(
    adata=model.adata,
    color="Enpp2",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Enpp2",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[2, 1],
    show=False)
plt.show()
fig.savefig(f"{figure_folder_path}/rps19_gp_genes_peaks.svg",
            bbox_inches="tight")

fig, axs = plt.subplots(nrows=6,
                        ncols=2,
                        figsize=(25, 25))
color_map = "RdGy_r"
sc.pl.spatial(
    adata=model.adata,
    color="Nkx6-2_TF_target_genes_GP",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Nkx6-2 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)    
color_map = "RdPu"
sc.pl.spatial(
    adata=model.adata,
    color="Plp1",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Plp1",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[0, 1],
    show=False)
sc.pl.spatial(
    adata=model.adata,
    color="Mbp",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Mbp",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[1, 0],
    show=False)
plt.show()
fig.savefig(f"{figure_folder_path}/nkx6-2_gp_genes_peaks.svg",
            bbox_inches="tight")

fig, axs = plt.subplots(nrows=6,
                        ncols=2,
                        figsize=(25, 25))
color_map = "RdGy_r"
sc.pl.spatial(
    adata=model.adata,
    color="Nkx2-2_TF_target_genes_GP",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Nkx2-2 GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)    
color_map = "RdPu"
sc.pl.spatial(
    adata=model.adata,
    color="Ermn",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Ermn",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[0, 1],
    show=False)
sc.pl.spatial(
    adata=model.adata,
    color="Sirt2",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Sirt2",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[1, 0],
    show=False)
sc.pl.spatial(
    adata=model.adata,
    color="Arx",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Arx",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[1, 1],
    show=False)
plt.show()
fig.savefig(f"{figure_folder_path}/nkx2-2_gp_genes_peaks.svg",
            bbox_inches="tight")

fig, axs = plt.subplots(nrows=6,
                        ncols=2,
                        figsize=(25, 25))
model.adata.obs["Olig2_TF_target_genes_GP_sign_corrected"] = -1 * model.adata.obs["Olig2_TF_target_genes_GP"]
color_map = "RdGy_r"
sc.pl.spatial(
    adata=model.adata,
    color="Olig2_TF_target_genes_GP_sign_corrected",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Olig2  GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)    
color_map = "RdPu"
sc.pl.spatial(
    adata=model.adata,
    color="Olig1",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Olig1",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[0, 1],
    show=False)
sc.pl.spatial(
    adata=model.adata,
    color="Sox10",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Sox10",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[1, 0],
    show=False)
sc.pl.spatial(
    adata=model.adata,
    color="Olig2",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Olig2",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[1, 1],
    show=False)
plt.show()
fig.savefig(f"{figure_folder_path}/olig2_gp_genes_peaks.svg",
            bbox_inches="tight")

fig, axs = plt.subplots(nrows=6,
                        ncols=2,
                        figsize=(25, 25))
model.adata.obs["Cd82_ligand_receptor_target_gene_GP_sign_corrected"] = -1 * model.adata.obs["Cd82_ligand_receptor_target_gene_GP"]
color_map = "RdGy_r"
sc.pl.spatial(
    adata=model.adata,
    color="Cd82_ligand_receptor_target_gene_GP_sign_corrected",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Cd82  GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)    
color_map = "RdPu"
sc.pl.spatial(
    adata=model.adata,
    color="Cd82",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Cd82",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[0, 1],
    show=False)
sc.pl.spatial(
    adata=model.adata,
    color="Gsn",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Gsn",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[1, 0],
    show=False)
sc.pl.spatial(
    adata=model.adata,
    color="Gfra1",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Gfra1",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[1, 1],
    show=False)
sc.pl.spatial(
    adata=model.adata,
    color="Tcf7l2",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Tcf7l2",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[2, 0],
    show=False)
sc.pl.spatial(
    adata=model.adata,
    color="Zfp36l1",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Zfp36l1",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[2, 1],
    show=False)
plt.show()
fig.savefig(f"{figure_folder_path}/cd82_gp_genes_peaks.svg",
            bbox_inches="tight")

fig, axs = plt.subplots(nrows=6,
                        ncols=2,
                        figsize=(25, 25))
color_map = "RdGy_r"
sc.pl.spatial(
    adata=model.adata,
    color="Nfasc_ligand_receptor_target_gene_GP",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Nfasc  GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)    
color_map = "RdPu"
sc.pl.spatial(
    adata=model.adata,
    color="Nfasc",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Nfasc",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[0, 1],
    show=False)
sc.pl.spatial(
    adata=model.adata,
    color="Mgll",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Mgll",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[1, 0],
    show=False)
sc.pl.spatial(
    adata=model.adata,
    color="Enpp2",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Enpp2",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[1, 1],
    show=False)
plt.show()
fig.savefig(f"{figure_folder_path}/nfasc_gp_genes_peaks.svg",
            bbox_inches="tight")

fig, axs = plt.subplots(nrows=6,
                        ncols=2,
                        figsize=(25, 25))
model.adata.obs["Glycine_metabolite_enzyme_sensor_GP_sign_corrected"] = -1 * model.adata.obs["Glycine_metabolite_enzyme_sensor_GP"]
color_map = "RdGy_r"
sc.pl.spatial(
    adata=model.adata,
    color="Glycine_metabolite_enzyme_sensor_GP_sign_corrected",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Glycine  GP",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)    
color_map = "RdPu"
sc.pl.spatial(
    adata=model.adata,
    color="Lap3",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Lap3",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[0, 1],
    show=False)
sc.pl.spatial(
    adata=model.adata,
    color="Gatm",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Gatm",
    legend_loc=None,
    colorbar_loc="bottom",
    use_raw=False,
    ax=axs[1, 0],
    show=False)
plt.show()
fig.savefig(f"{figure_folder_path}/glycine_gp_genes_peaks.svg",
            bbox_inches="tight")

In [ ]:
### Supplementary Fig. 26b: Corpus Callosum niche: enriched de novo gps ###
fig, axs = plt.subplots(nrows=5,
                        ncols=2,
                        figsize=(25, 25))
color_map = "RdGy_r"
sc.pl.spatial(
    adata=model.adata,
    color="Add-on_89_GP",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Add-on 89",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 0],
    show=False)    
color_map = "RdPu"
sc.pl.spatial(
    adata=model.adata,
    color="Mobp",
    color_map=color_map,
    spot_size=spot_size,
    title=f"Mobp",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[0, 1],
    show=False)
sc.pl.spatial(
    adata=model.adata,
    color="Tspan2",
    use_raw=False,
    color_map=color_map,
    spot_size=spot_size,
    title=f"Tspan2",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 0],
    show=False)
sc.pl.spatial(
    adata=model.adata,
    color="Pllp",
    use_raw=False,
    color_map=color_map,
    spot_size=spot_size,
    title="Pllp",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[1, 1],
    show=False)
sc.pl.spatial(
    adata=model.adata,
    color="Trf",
    use_raw=False,
    color_map=color_map,
    spot_size=spot_size,
    title="Trf",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[2, 0],
    show=False)
sc.pl.spatial(
    adata=model.adata,
    color="Pdlim2",
    use_raw=False,
    color_map=color_map,
    spot_size=spot_size,
    title="Pdlim2",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[2, 1],
    show=False)
sc.pl.spatial(
    adata=model.adata_atac,
    color="chr1:24612403-24613300",
    use_raw=False,
    color_map=color_map,
    spot_size=spot_size,
    title=f"chr1:24612403-24613300",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[3, 0],
    show=False)
sc.pl.spatial(
    adata=model.adata_atac,
    color="chr1:172349675-172350564",
    use_raw=False,
    color_map=color_map,
    spot_size=spot_size,
    title="chr1:172349675-172350564",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[3, 1],
    show=False)
sc.pl.spatial(
    adata=model.adata_atac,
    color="chr4:48629122-48629953",
    use_raw=False,
    color_map=color_map,
    spot_size=spot_size,
    title="chr4:48629122-48629953",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[4, 0],
    show=False)
sc.pl.spatial(
    adata=model.adata_atac,
    color="chr14:54935868-54936644",
    use_raw=False,
    color_map=color_map,
    spot_size=spot_size,
    title="chr14:54935868-54936644",
    legend_loc=None,
    colorbar_loc="bottom",
    ax=axs[4, 1],
    show=False)
plt.show()
fig.savefig(f"{figure_folder_path}/addon89_gp_genes_peaks.svg",
            bbox_inches="tight")

### 3.2 Save Results

In [ ]:
# Log normalize counts for cellxgene server
sc.pp.normalize_total(model.adata, target_sum=1e4)
sc.pp.log1p(model.adata)
    
# Store gp summary in adata
gp_summary = model.get_gp_summary()
for col in gp_summary.columns:
    gp_summary[col] = gp_summary[col].astype(str)
model.adata.uns["nichecompass_gp_summary"] = gp_summary

model.adata.write(f"{result_folder_path}/{dataset}_analysis.h5ad")